In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
import pandas_datareader as web
from scipy.stats import norm

In [2]:
#Get the price data
ticker = "GOOG"
start = '2022-08-29'
end = '2022-11-28'
goog = pd.DataFrame(web.get_data_yahoo(ticker, start=start, end=end)['Close'])
mondays = [(i+1)*7 for i in range(13)]
start = datetime.strptime(start, '%Y-%m-%d')
mondays = pd.DataFrame([start]+[start + timedelta(days=i) for i in mondays], columns=['Date'])
data = goog.merge(mondays, on='Date', how='outer')
#have to add september 2nd data as the 5th because of the holiday
data = data.sort_values(by='Date').fillna(method="ffill")
data = data.merge(mondays, on='Date', how='inner')

In [3]:
#Setup
print('The stock I chose is Google with ticker', ticker,".")
print("The price at close on 08-29-2022 is $",np.round(data.iloc[0,1],decimals=3),".")
data

The stock I chose is Google with ticker GOOG .
The price at close on 08-29-2022 is $ 110.34 .


,Date,Close
0,2022-08-29,110.339996
1,2022-09-05,108.680000
2,2022-09-12,111.870003
3,2022-09-19,103.849998
4,2022-09-26,98.809998
5,2022-10-03,99.300003
6,2022-10-10,98.709999
7,2022-10-17,100.779999
8,2022-10-24,102.970001
9,2022-10-31,94.660004


In [4]:
#Get VIX data
vix = web.DataReader("VIXCLS", "fred", start, end)
vix['Date'] = vix.index
vix = vix.merge(mondays, on='Date', how='inner')

In [5]:
#1 One put option
E = data.iloc[0,1]*.99
T = 13/52
S0 = data.iloc[0,1]
sigma = vix['VIXCLS'][:1]/100
r = 0.003

d1 = (np.log(S0/E)+(r+(sigma**2)/2)*(T))/(sigma*np.sqrt(T))
d2 = (np.log(S0/E)+(r-(sigma**2)/2)*(T))/(sigma*np.sqrt(T))
Put = (E*np.e**(-r*T))*norm.cdf(-d2)-S0*norm.cdf(-d1)
Put1 = np.round(Put,decimals=3)
print("The annual risk free rate is 0.3% and the VIX on 08-29-2022 is", vix['VIXCLS'][:1][0],".")
print('Using Black-Scholes, the price of one put option for Goog at time t=0 is $',Put1[0],".")

The annual risk free rate is 0.3% and the VIX on 08-29-2022 is 26.21 .
Using Black-Scholes, the price of one put option for Goog at time t=0 is $ 5.16 .


In [6]:
#2 Set up a hedged portfolio

#For t=0, T-t=13
data['T-t']=13
#Create two new variables to fill in
data['Delta_Shares']=0
data['Cash']=0
# r_delta_t is the risk-free rate mutiplied by a time period of one week. This is used to calculate the interest earned on the cash in the portfolio each week
r_delta_t=(1/52)*r
#Get the value for T-t and how many shares to sell to hedge one Put with
# -norm.cdf(-d1) is used for Delta_Shares
for i in range(13):
    data.iloc[i,2]=(data.iloc[i,2]-i)/52
    data.iloc[i,3]=-norm.cdf(-((np.log(data.iloc[i,1]/E)+(r+(sigma**2)/2)*(data.iloc[i,2]))/(sigma*np.sqrt(data.iloc[i,2]))))
#Time T-t=0 at expiry
data.iloc[13,2]=0
#Calculate the cash in the portfolio at time t=0
data.iloc[0,4]=((Put*100)+(-data.iloc[0,3]*S0*100))
#Calculate the cash in the portfolio each week after t=0
#Next you add the interest earned on the cash in the portfolio for each week by multiplying the cash in the portfolio by e^r_delta_t 
#Then you find the amount of shares that need to be bought or sold and multiply that by that week's close price(by 100 for 100 puts)
for i in range(13):
    a=i+1
    data.iloc[a,4]=(data.iloc[i,4]*np.e**r_delta_t)+(-(data.iloc[a,3]-data.iloc[i,3])*data.iloc[a,1]*100)
#Calculate the interest earned on the cash in the portfolio in the last week
data.iloc[13,4]=data.iloc[12,4]*np.e**r_delta_t
#Carry over the amount of shares hedged in the last week
data.iloc[13,3]=data.iloc[12,3]
#Calculate the final balance of the portfolio by 
#subtracting the amount it costs to buy back the hedged shares at the last close price
#and the final payoff of the 100 puts from the cash in the portfolio
final_balance=data.iloc[13,4]-(100*(E-data.iloc[13,1]))+(data.iloc[13,3]*100*data.iloc[13,1])
print('The following is the data by week for the hedged portfolio. Delta_Shares is per Put contract.')
data
print("Using a delta-neutral hedging procedure, the value of the hedged portfolio at expiry is $",np.round(final_balance,decimals=3),".")

The following is the data by week for the hedged portfolio. Delta_Shares is per Put contract.
Using a delta-neutral hedging procedure, the value of the hedged portfolio at expiry is $ -106.515 .


In [7]:
#3 Set up an unhedged portfolio

#Multiply the cash gained from selling 100 puts by e^T*r
cash = (100*Put)*np.e**((13/52)*r)
#The final balance of the unhedged portfolio is calculated by subtracting the payoff of the puts from the cash in the portfolio at expiry
final_balance2 = cash-100*(data.iloc[0,1]-data.iloc[12,1])
print('The unhedged portfolio value at expirey is $', np.round(final_balance2[0],decimals=3),'.')

The unhedged portfolio value at expirey is $ -934.57 .


In [8]:
#4 Compare the hedged portfolio to the unhedged portfolio
if final_balance>final_balance2: 
    print("The hedged portfolio has a higher value at expiry and is therefore favorable.")
elif final_balance2>final_balance:
    print("The unhedged portfolio has a higher value at expiry and is therefore favorable.")

The hedged portfolio has a higher value at expiry and is therefore favorable.


In [9]:
#5 compare historical volatility vs VIX in a hedged portfolio
data2 = goog.merge(mondays, on='Date', how='outer')
data2 = data2.sort_values(by='Date').fillna(method="ffill")
data2 = data2.merge(mondays, on='Date', how='inner')
#Calculate the log return each week
data2['log_return']=0
for i in range(13):
    a=i+1
    data2.iloc[a,2]=np.log(data2.iloc[a,1]/data2.iloc[i,1])
#The volatility is calculated using the equation 6.6.1 in the notes
vol = (1/np.sqrt(13/52))*(np.sqrt((1/14)*sum(data2['log_return']**2)-(1/(13*14))*(sum(data2['log_return'])**2)))
#Follow the same procedure as #1 but with the new volatility input
E = data2.iloc[0,1]*.99
T = 13/52
S0 = data2.iloc[0,1]
sigma = vol
r = 0.003

d1 = (np.log(S0/E)+(r+(sigma**2)/2)*(T))/(sigma*np.sqrt(T))
d2 = (np.log(S0/E)+(r-(sigma**2)/2)*(T))/(sigma*np.sqrt(T))
Put2 = (E*np.e**(-r*T))*norm.cdf(-d2)-S0*norm.cdf(-d1)
print('Imputing the actual volitility into the Black-Scholes model, the price of one put option for Goog at time t=0 is $',np.round(Put2,decimals=3),'.')
#Follow the same delta-neutral hedging procedure at #2
data2['T-t']=13
data2['Delta_Shares']=0
data2['Cash']=0
r_delta_t=(1/52)*r
for i in range(13):
    data2.iloc[i,3]=(data2.iloc[i,3]-i)/52
    data2.iloc[i,4]=-norm.cdf(-((np.log(data2.iloc[i,1]/E)+(r+(sigma**2)/2)*(data2.iloc[i,3]))/(sigma*np.sqrt(data2.iloc[i,3]))))
data2.iloc[13,3]=0
data2.iloc[0,5]=((Put2*100)+(-data2.iloc[0,4]*S0*100))
for i in range(13):
    a=i+1
    data2.iloc[a,5]=(data2.iloc[i,5]*np.e**r_delta_t)+(-(data2.iloc[a,4]-data2.iloc[i,4])*data2.iloc[a,1]*100)
data2.iloc[13,5]=data2.iloc[12,5]*np.e**r_delta_t
data2.iloc[13,4]=data2.iloc[12,4]
final_balance3=data2.iloc[13,5]-(100*(E-data2.iloc[13,1]))+(data2.iloc[13,4]*100*data2.iloc[13,1])
print("Using the actual volatility and a delta-neutral hedging procedure, the value of the portfolio at expiry is $",np.round(final_balance3,decimals=3),".")
if final_balance>final_balance3:
    print("It would be better to use the VIX model for hedging.") 
    print("The original hedged portfolio has a higher value at expiry than the hedged portfolio using actual volatility.")
elif final_balance3>final_balance:
    print("It would be better to use the model with the actual volitility for hedging.") 
    print("The original hedged portfolio has a lower value at expiry than the hedged portfolio using actual volatility.")
print('The following is the data by week for the hedged portfolio using actual volitility.')
print('Delta_Shares is per Put contract.')
data2

Imputing the actual volitility into the Black-Scholes model, the price of one put option for Goog at time t=0 is $ 1.367 .
Using the actual volatility and a delta-neutral hedging procedure, the value of the portfolio at expiry is $ -349.677 .
It would be better to use the VIX model for hedging.
The original hedged portfolio has a higher value at expiry than the hedged portfolio using actual volatility.
The following is the data by week for the hedged portfolio using actual volitility.
Delta_Shares is per Put contract.


,Date,Close,log_return,T-t,Delta_Shares,Cash
0,2022-08-29,110.339996,0.000000,0.250000,-0.393406,4477.516118
1,2022-09-05,108.680000,-0.015159,0.230769,-0.533870,6004.332189
2,2022-09-12,111.870003,0.028930,0.211538,-0.263601,2981.182744
3,2022-09-19,103.849998,-0.074390,0.192308,-0.902117,9612.346915
4,2022-09-26,98.809998,-0.049749,0.173077,-0.996968,10550.124362
5,2022-10-03,99.300003,0.004947,0.153846,-0.997192,10552.954078
6,2022-10-10,98.709999,-0.005959,0.134615,-0.999188,10573.270558
7,2022-10-17,100.779999,0.020754,0.115385,-0.996586,10547.651871
8,2022-10-24,102.970001,0.021498,0.096154,-0.984988,10428.840796
9,2022-10-31,94.660004,-0.084146,0.076923,-1.000000,10571.542375
